In [2]:
import numpy as np
from scipy import stats
from astropy.table import join,Table,Column,vstack

dat = Table.read('DistancesFramesv14.fits')

dat.info('stats')
  



<Table length=339693>
    name         mean        std         min         max    
------------ ----------- ----------- ----------- -----------
      CATAID 1.77872e+06 1.95179e+06        6648 9.10013e+06
          RA      190.91     86.0915     30.0452     352.396
         DEC    -5.86577     12.0544    -35.4997      3.4998
          NQ     3.99923    0.535127           2           5
     Z_HELIO    0.254002     0.28539    -0.00967     7.02314
        Z_LG     0.25361    0.285255  -0.0102599     7.01825
       Z_CMB    0.254633    0.285685 -0.00848463     7.03287
     Z_TONRY    0.254584    0.285728           0     7.03287
Z_TONRY_HIGH    0.255315    0.285919           0     7.03769
 Z_TONRY_LOW    0.253878    0.285515           0     7.02805
 DM_70_30_70     36.7395     11.3548           0      49.204
DM_100_30_70     36.0312     11.1406           0     48.4295
 DM_70_25_75     36.7587     11.3624           0     49.3397
DM_100_25_75     36.0504     11.1483           0     48.5652


In [3]:
def  z_tozLG(dat):
    """Transform redshift from Heliocentric to Local Group velocity frame.
    """
    
    #Velocity of the sun baryocentre in the Local Group Frame
    v_LG=316.0 # km/s  NED and AJ 111, 794, 1996
    RA_LG=np.deg2rad(325.371)  # l=93deg b=-4deg also from NED converted to RA,DEC by NED
    DEC_LG=np.deg2rad(47.527)
    #Projection of this velocity along object RA and dec np.deg2rad(
    v_proj=v_LG*(np.sin(np.deg2rad(dat['DEC']))*np.sin(DEC_LG)+np.cos(np.deg2rad(dat['DEC']))*np.cos(DEC_LG)*np.cos(np.deg2rad(dat['RA'])-RA_LG))
    #Compute chnage to redshift and compare to value given in GAMA catalogue
    dat.add_column(Column(name='ZLG', data= (1.0+dat['Z_HELIO'])*(1.0+v_proj/(299792.458))-1.0 ))
    dat.add_column(Column(name='Zdiff', data=(dat['ZLG']-dat['Z_LG'])))
    return

In [4]:
z_tozLG(dat)
dat.info('stats')

<Table length=339693>
    name         mean         std         min          max    
------------ ------------ ----------- ------------ -----------
      CATAID  1.77872e+06 1.95179e+06         6648 9.10013e+06
          RA       190.91     86.0915      30.0452     352.396
         DEC     -5.86577     12.0544     -35.4997      3.4998
          NQ      3.99923    0.535127            2           5
     Z_HELIO     0.254002     0.28539     -0.00967     7.02314
        Z_LG      0.25361    0.285255   -0.0102599     7.01825
       Z_CMB     0.254633    0.285685  -0.00848463     7.03287
     Z_TONRY     0.254584    0.285728            0     7.03287
Z_TONRY_HIGH     0.255315    0.285919            0     7.03769
 Z_TONRY_LOW     0.253878    0.285515            0     7.02805
 DM_70_30_70      36.7395     11.3548            0      49.204
DM_100_30_70      36.0312     11.1406            0     48.4295
 DM_70_25_75      36.7587     11.3624            0     49.3397
DM_100_25_75      36.0504     11.

In [ ]:
def z_to_gsr(dat, v_sun=None):
    """Transform a barycentric redshift to the Galactic Standard of Rest
    (GSR).

    """
    if v_sun is None:
        v_sun = coord.Galactocentric().galcen_v_sun.to_cartesian()
   
    
    #dummy distance and proper motion arrays
    dis=np.ones(dat['Z'].size)
    pm=np.zeros(dat['Z'].size)
    v_r=np.zeros(dat['Z'].size)# zero radial velocity and proper motion is important
    
    # Use RA and Dec to set up a coordinates in the ICRS frame
    c = coord.SkyCoord(
    ra=dat['RA'] * u.degree,
    dec=dat['DEC'] * u.degree,
    distance=dis * u.pc,
    pm_ra_cosdec=pm * u.mas / u.yr,
    pm_dec=pm* u.mas / u.yr,
    radial_velocity=v_r * u.km / u.s,
    frame="icrs",
)
    
        
    # convert to galactic coordinates    
    gal = c.transform_to(coord.Galactic)
    cart_data = gal.data.to_cartesian()
    unit_vector = cart_data / cart_data.norm()

    #
    v_proj = v_sun.dot(unit_vector) # projection of suns' velocity in galactocentric frame along object direction
    # compute effect on redshift
    dat.add_column(Column(name='Zgal', data= (1.0+dat['Z'])*(1.0+v_proj/(299792.458 * u.km / u.s))-1.0 ))
    return 



In [ ]:
z_to_gsr(dat)

In [ ]:
dat.info('stats')